<a href="https://colab.research.google.com/github/c-e-leiva/procesamientoDelHabla/blob/main/Desafio20NewsGroupsNaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups**

## **Consignas:**

**1**. **Vectorizar documentos**. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.


**2**. **Entrenar modelos de clasificación Naïve Bayes** para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. **Transponer la matriz documento-término**. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.



---


---



---




# 1. Vectorizar documentos y medir similaridad

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

# Cargar el dataset 20 newsgroups (subset train)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# Instanciar el vectorizador TF-IDF
tfidfvect = TfidfVectorizer()

# Fit-transformar los datos de entrenamiento
X_train = tfidfvect.fit_transform(newsgroups_train.data)

# Obtener el vocabulario
vocabulary = tfidfvect.vocabulary_

# Mostrar algunas palabras del vocabulario
words = list(vocabulary.keys())
print(f'Número total de palabras en el vocabulario: {len(words)}')
print('Algunas palabras en el vocabulario:', words[:30])  # Muestra las primeras 10 palabras

Número total de palabras en el vocabulario: 101631
Algunas palabras en el vocabulario: ['was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'door', 'sports', 'looked', 'to', 'be', 'from', 'late', '60s', 'early', '70s', 'called', 'bricklin', 'doors']


In [30]:
#Vectorización de documentos

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
import numpy as np

# Cargar el dataset 20 newsgroups (subset train)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# Instanciar el vectorizador TF-IDF
tfidfvect = TfidfVectorizer()

# Fit-transformar los datos de entrenamiento
X_train = tfidfvect.fit_transform(newsgroups_train.data)

# Guardar las etiquetas de clasificación
y_train = newsgroups_train.target

#Tomamos 5 documentos al azar
# Definimos un array con 5 índices de documentos aleatorios
np.random.seed(111)  # Para reproducibilidad
random_docs = np.random.choice(X_train.shape[0], 5, replace=False)

#Medir similaridad con el resto de los documentos

for idx in random_docs:
    print(f"Documento {idx}:\n")

    # Mostrar el contenido del documento (solo los primeros 100 caracteres para tener una idea)
    print(newsgroups_train.data[idx][:100], "\n")

    # Calcular la similaridad coseno entre el documento seleccionado y todos los demás
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Obtener los índices de los 5 documentos más similares (excluyendo el propio)
    mostsim = np.argsort(cossim)[::-1][1:6]

    # Mostrar la clase del documento original
    print(f"Clase del documento original: {newsgroups_train.target_names[y_train[idx]]}\n")

    # Mostrar las clases de los 5 documentos más similares
    for i in mostsim:
        print(f"Documento similar {i}:")
        print(f"Clase: {newsgroups_train.target_names[y_train[i]]}")
        print(f"Contenido (primeros 300 caracteres): {newsgroups_train.data[i][:100]}\n")

    print("="*80)

Documento 8917:

I am running Windows 3.1, Windows for Work Groups and just loaded Dos 6.

What`s happening appears a 

Clase del documento original: comp.os.ms-windows.misc

Documento similar 2140:
Clase: comp.os.ms-windows.misc
Contenido (primeros 300 caracteres): In comp.os.ms-windows.misc you write:


you might want to look in windows FAQ for this one, but here

Documento similar 3451:
Clase: comp.os.ms-windows.misc
Contenido (primeros 300 caracteres): Hi there,
I'm having a bizarre video problem within Windows 3.1.  I have a 286 with
a GVGA-16 video 

Documento similar 8907:
Clase: comp.os.ms-windows.misc
Contenido (primeros 300 caracteres): Thanks to all those people who recommended Workspace managers for
Windows 3.1.  I found 3 shareware 

Documento similar 2391:
Clase: comp.os.ms-windows.misc
Contenido (primeros 300 caracteres): I quit windows normally to run a special DOS app, got done with it
and tried to start windows.  Ok g

Documento similar 8832:
Clase: comp.windows.x
Con

# 2. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test.

 Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [31]:
#Entrenamiento de modelos de clasificación Naïve Bayes

from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

#Entrenamiento del modelo Naïve Bayes Multinomial**
clf_multinomial = MultinomialNB()
clf_multinomial.fit(X_train, y_train)

#Vectorización de los datos de prueba
# Cargamos el dataset de test
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# Transformamos los datos de prueba usando el vectorizador TF-IDF
X_test = tfidfvect.transform(newsgroups_test.data)

#Predicción y cálculo del F1-score para MultinomialNB
y_pred_multinomial = clf_multinomial.predict(X_test)
f1_score_multinomial = f1_score(newsgroups_test.target, y_pred_multinomial, average="macro")
print(f'F1-score (macro) MultinomialNB: {f1_score_multinomial}')

#Entrenamiento del modelo Naïve Bayes Complement
clf_complement = ComplementNB()
clf_complement.fit(X_train, y_train)

#Predicción y cálculo del F1-score para ComplementNB
y_pred_complement = clf_complement.predict(X_test)
f1_score_complement = f1_score(newsgroups_test.target, y_pred_complement, average="macro")
print(f'F1-score (macro) ComplementNB: {f1_score_complement}')

F1-score (macro) MultinomialNB: 0.5854345727938506
F1-score (macro) ComplementNB: 0.692953349950875





# **3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [32]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Transponer la matriz documento-término
X_train_transposed = X_train.T

# Definir las 5 palabras de interés
palabras_interes = ['health', 'activity', 'nutrition', 'balance', 'income']

# Evaluar la similitud de las palabras de interés
for palabra in palabras_interes:
    if palabra in tfidfvect.vocabulary_:
        idx = tfidfvect.vocabulary_[palabra]
        idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}
        palabra_vect = X_train_transposed[idx]
        palabras_similares = cosine_similarity(palabra_vect.reshape(1, -1), X_train_transposed)[0]
        idx_mas_similares = np.argsort(palabras_similares)[::-1][1:6]
        print(f'Palabra elegida: {palabra}')
        for idx in idx_mas_similares:
            print(f'Palabra similar encontrada: {idx2word[idx]} (similaridad: {palabras_similares[idx]:.4f})')
    else:
        print(f'La palabra "{palabra}" no está en el vocabulario.')


Palabra elegida: health
Palabra similar encontrada: ohip (similaridad: 0.3304)
Palabra similar encontrada: provincial (similaridad: 0.2998)
Palabra similar encontrada: care (similaridad: 0.2824)
Palabra similar encontrada: traditionalists (similaridad: 0.2799)
Palabra similar encontrada: fiscally (similaridad: 0.2799)
Palabra elegida: activity
Palabra similar encontrada: lifeguards (similaridad: 0.2819)
Palabra similar encontrada: rollerblade (similaridad: 0.2819)
Palabra similar encontrada: homily (similaridad: 0.2819)
Palabra similar encontrada: rugby (similaridad: 0.2819)
Palabra similar encontrada: seatbelts (similaridad: 0.2819)
Palabra elegida: nutrition
Palabra similar encontrada: harperperennial (similaridad: 0.7458)
Palabra similar encontrada: 272007 (similaridad: 0.7458)
Palabra similar encontrada: gershoff (similaridad: 0.7458)
Palabra similar encontrada: barbecued (similaridad: 0.6254)
Palabra similar encontrada: unrealistically (similaridad: 0.5813)
Palabra elegida: balanc



---

---

---





# Análisis de Texto con 20 Newsgroups

En este proyecto, se utilizó el conjunto de datos **20 Newsgroups**, donde se aplicó la **vectorización** de documentos para convertir texto en representaciones numéricas, eliminando elementos no relevantes como encabezados, pies de página y citas. Se generó un vocabulario de **101,631 palabras**.

Para asegurar la reproducibilidad, se definió una **semilla aleatoria** y se seleccionaron 5 documentos aleatorios para medir la **similaridad** con el resto del conjunto mediante **similaridad coseno**. Se identificaron los **5 documentos más similares**, junto con sus respectivas clases.

Luego, se entrenaron modelos de clasificación utilizando **Naïve Bayes**, evaluando su desempeño con el **F1-score macro** en el conjunto de prueba. Se compararon los modelos **MultinomialNB** y **ComplementNB**, obteniendo un F1-score de **0.5854** para el primero y **0.6930** para el segundo, indicando que **ComplementNB** tuvo un mejor desempeño.

Finalmente, se transpuso la matriz documento-término para obtener una matriz término-documento. Se eligieron **5 palabras de interés** ('health', 'activity', 'nutrition', 'balance', 'income') para estudiar su **similaridad**, analizando las 5 palabras más similares a cada una.

## Reflexiones y Aprendizajes

Esta experiencia ha sido especialmente interesante para mí, ya que me ha permitido aplicar técnicas de procesamiento de lenguaje natural y explorar modelos de clasificación de texto. Los resultados obtenidos, especialmente el desempeño del modelo **ComplementNB**, subrayan la importancia de elegir el modelo adecuado para mejorar la precisión en las clasificaciones.

Además, esta práctica ha reforzado mi interés por seguir aprendiendo en el campo del análisis de datos y el procesamiento de texto. Reconozco que hay mucho más por descubrir y perfeccionar en este ámbito, y estoy motivado para continuar explorando nuevas metodologías y herramientas que me permitan mejorar mis habilidades y conocimientos.
